In [1]:
#| default_exp trainer

In [1]:
#| hide
from ipynb_path import *

In [2]:
#| export
from __future__ import annotations
import jax, jax.numpy as jnp, jax.random as jrand
import haiku as hk
import optax
import chex
from dataclasses import dataclass
import functools as ft
from typing import Callable, Tuple, Any, Sequence, Iterable, Mapping, Dict, List, NamedTuple
import copy

## Trainer

In [ ]:
#| export
class TrainState(NamedTuple):
    epoch: int
    step: int
    params: hk.Params
    state: hk.State
    opt_state: optax.OptState
    next_key: jrand.PRNGKey
    logs: dict = None

In [99]:
#| export
@dataclass
class Trainer:
    transformed: hk.TransformedWithState | hk.MultiTransformedWithState
    optimizers: optax.GradientTransformation | Sequence[optax.GradientTransformation]
    rng_key: jrand.PRNGKey = None

    # callback functions
    callbacks: List[Any] = None
    train_step_fn: TrainStepFn = None
    val_step_fn: ValStepFn = None

    # trainer configs
    lr: float = 1e-3
    n_epochs: int = 1

    # model train state
    _train_state: TrainState = None

    @property
    def train_state(self):
        return self._train_state

    def _initialize_key(self):
        if self.rng_key is None:    return jrand.PRNGKey(42) # TODO: use global
        else:                       return self.rng_key

    def _initialize_model_param_and_state(
        self, key: jrand.PRNGKey, shape: Sequence[int]
    ):
        params, state = self.transformed.init(key, jnp.zeros(shape))
        return params, state

    def _initialize_opt_state(self, params: hk.Params):
        # if isinstance(self.optimizers, Sequence):
        #     return [opt.init(params) for opt in self.optimizers]
        if isinstance(self.optimizers, optax.GradientTransformation):
            return self.optimizers.init(params) 
        else:
            raise ValueError(f"Invalid optimizers. Expected `optax` optimizers.")
    
    # def _initialize_step_fns(self):
    #     if self.train_step_fn is None:
    #         self.train_step_fn = TrainStepFn(self)

    #     if self.val_step_fn is None:
    #         self.val_step_fn = ValStepFn(self)

    def _initialize(self, batch: Tuple[jax.Array, ...]):
        xs_shape = batch[0].shape
        key1, next_key = jrand.split(self._initialize_key())
        params, state = self._initialize_model_param_and_state(key1, shape=xs_shape)
        opt_states = self._initialize_opt_state(params)
        self._train_state = TrainState(
            epoch=0, step=0,
            params=params, state=state, 
            opt_state=opt_states,
            next_key=next_key,
        )
        # self._initialize_step_fns()
        
    def _run_callbacks(self):
        if self.callbacks is not None:
            for callback in self.callbacks:
                callback(self.train_state)

    def update_train_state(self, train_state: TrainState = None, **kwargs):
        if train_state is None and kwargs == {}:
            raise ValueError("Either `train_state` or `kwargs` must be provided.")
        if train_state is None:
            train_state = self.train_state._replace(**kwargs)
        self._train_state = train_state

    def train_step(self, batch: Tuple[jax.Array, ...]) -> None:
        if self.train_step_fn is None:
            self.train_step_fn = TrainStepFn(self)
        upt_train_state = self.train_step_fn(self.train_state, batch)
        self.update_train_state(upt_train_state)
    
    def val_step(self, batch: Tuple[jax.Array, ...]) -> None:
        if self.val_step_fn is None:
            self.val_step_fn = ValStepFn(self)
        upt_train_state = self.val_step_fn(self.train_state, batch)
        self.update_train_state(upt_train_state)

    def validate_train_step(self, batch: Tuple[jax.Array, ...]):
        cur_train_state = copy.deepcopy(self.train_state)
        self.train_step(batch)
        if cur_train_state.step == self.train_state.step:
            raise ValueError("Train state is not updated after `train_step`.")

    def fit(self, train_dataloader, val_dataloader=None):
        cur_steps = 0
        for epoch in range(self.n_epochs):
            for batch in train_dataloader:
                if self.train_state is None:
                    self._initialize(batch)
                    self.validate_train_step(batch)
                else:
                    self.train_step(batch)
                self._run_callbacks()
                cur_steps += 1
                assert cur_steps == self.train_state.step

            if val_dataloader is not None:
                for batch in val_dataloader:
                    self.val_step(self.train_state, batch)
                    cur_steps += 1
                    assert cur_steps == self.train_state.step

            self.update_train_state(epoch=epoch+1)


## Step Functions

In [ ]:
#| export
class StepFn:
    def __init__(self, trainer: Trainer, *args, **kwargs) -> None:
        self._trainer = trainer

    @property
    def trainer(self): return self._trainer

    @property
    def forward(self): return self.trainer.transformed
    
    @property
    def optimizer(self): return self.trainer.optimizers

In [93]:
#| export
class TrainStepFn(StepFn):
    loss_fn: Callable[[jax.Array, jax.Array], float] = None
    
    def __init__(self, trainer: Trainer, loss_fn=None) -> None:
        super().__init__(trainer)
        if loss_fn is None:
            loss_fn = optax.softmax_cross_entropy_with_integer_labels
        self.loss_fn = loss_fn

    @ft.partial(jax.jit, static_argnums=(0,))
    def __call__(self, train_state: TrainState, batch: Tuple[jax.Array, ...]):
        def loss_fn(params: hk.Params):
            logits, new_state = self.forward.apply(
                params, state,
                rng_key, # <== rng
                inputs, is_training=True # <== inputs
            )
            loss = self.loss_fn(logits, labels).mean()
            return (loss, new_state)
        
        inputs, labels = batch
        rng_key, next_key = jrand.split(train_state.next_key)
        state = train_state.state
        (loss, new_state), grads = jax.value_and_grad(loss_fn, has_aux=True)(train_state.params)
        updates, new_opt_state = self.optimizer.update(
            grads, train_state.opt_state, train_state.params)
        new_params = optax.apply_updates(train_state.params, updates)
        return TrainState(
            epoch=train_state.epoch,
            step=train_state.step + 1,
            params=new_params,
            state=new_state,
            opt_state=new_opt_state,
            next_key=next_key,
            logs={'train/loss': loss}
        )

In [94]:
#| export
class ValStepFn(StepFn):
    metric_fns: Dict[str, Callable[[jax.Array, jax.Array], float]] = None

    def __init__(self, trainer: Trainer, metric_fns=None) -> None:
        super().__init__(trainer)
        if metric_fns is None:
            metric_fns = optax.softmax_cross_entropy_with_integer_labels
        self.metric_fns = metric_fns

    @ft.partial(jax.jit, static_argnums=(0,))
    def __call__(self, train_state: TrainState, batch: Tuple[jax.Array, ...]):        
        inputs, labels = batch
        rng_key, next_key = jrand.split(train_state.next_key)
        logits, _ = self.forward.apply(
            train_state.params, train_state.state,
            rng_key, # <== rng
            inputs, is_training=False # <== inputs
        )
        logs = {
            f'val/{name}': metric_fn(logits, labels)
            for name, metric_fn in self.metric_fns.items()
        }

        train_state = train_state._replace(
            step=train_state.step + 1,
            next_key=next_key, logs=logs
        )
        return train_state

## Test

### Fake Module

In [73]:
class LinearBatchNorm(hk.Module):
    "Linear layer with batch normalization"
    def __init__(self, output_size, name=None):
        super().__init__(name=name)
        self.output_size = output_size
    
    def __call__(self, x, training=False):
        return hk.BatchNorm(True, True, 0.9)(hk.Linear(self.output_size)(x), is_training=training)    

In [74]:
def make_hk_module(
    module: hk.Module, # haiku module 
    *args, # haiku module arguments
    **kargs, # haiku module arguments
) -> hk.Transformed:

    def model_fn(x, is_training: bool = True):
        return module(*args, **kargs)(x, is_training)
    
    return hk.transform_with_state(model_fn)

module = make_hk_module(LinearBatchNorm, 2)

### Fake Data

In [30]:
from jax_dataloader import DataLoader, ArrayDataset
from sklearn.datasets import make_classification

2023-06-25 15:08:49.637593: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [31]:
xs, ys = make_classification(n_samples=2000, n_features=10, random_state=0)
ds = ArrayDataset(xs, ys)
dl = DataLoader(ds, 'jax', batch_size=128)

### Training

In [95]:
trainer = Trainer(
    transformed=module,
    optimizers=optax.adam(1e-3),
    callbacks=[],
)
# model._initialize_param_and_state(jrand.PRNGKey(42), (10, 10))
# model._initialize_opt_state(jrand.PRNGKey(42))
# trainer._initialize((jnp.zeros((10, 10)), jnp.zeros((10, 10))))

In [96]:
trainer.fit(dl)